In [1]:
!pip install transformers
!pip install bitsandbytes
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 11.4 MB/s eta 0:00:0000:0100:01


In [2]:
from sklearn.model_selection import train_test_split

import transformers

import pandas as pd

import torch
import torch.nn.functional as F
from torch import nn
from torch.cuda.amp import custom_fwd, custom_bwd

from bitsandbytes.functional import quantize_blockwise, dequantize_blockwise

from tqdm.auto import tqdm

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /opt/conda/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/cuda/lib'), PosixPath('/usr/local/lib/x86_64-linux-gnu')}
  warn(msg)


**Quatization**

In [3]:
class FrozenBNBLinear(nn.Module):
    def __init__(self, weight, absmax, code, bias=None):
        assert isinstance(bias, nn.Parameter) or bias is None
        super().__init__()
        self.out_features, self.in_features = weight.shape
        self.register_buffer("weight", weight.requires_grad_(False))
        self.register_buffer("absmax", absmax.requires_grad_(False))
        self.register_buffer("code", code.requires_grad_(False))
        self.adapter = None
        self.bias = bias
 
    def forward(self, input):
        output = DequantizeAndLinear.apply(input, self.weight, self.absmax, self.code, self.bias)
        if self.adapter:
            output_cloned = torch.clone(output + self.adapter(input))
            return output_cloned
        else :
            return output
 
    @classmethod
    def from_linear(cls, linear: nn.Linear) -> "FrozenBNBLinear":
        weights_int8, state = quantize_blockise_lowmemory(linear.weight)
        return cls(weights_int8, *state, linear.bias)
 
    def __repr__(self):
        return f"{self.__class__.__name__}({self.in_features}, {self.out_features})"
 
 
class DequantizeAndLinear(torch.autograd.Function): 
    @staticmethod
    @custom_fwd
    def forward(ctx, input: torch.Tensor, weights_quantized: torch.ByteTensor,
                absmax: torch.FloatTensor, code: torch.FloatTensor, bias: torch.FloatTensor):
        weights_deq = dequantize_blockwise(weights_quantized, absmax=absmax, code=code)
        ctx.save_for_backward(input, weights_quantized, absmax, code)
        ctx._has_bias = bias is not None
        return F.linear(input, weights_deq, bias).clone()
 
    @staticmethod
    @custom_bwd
    def backward(ctx, grad_output: torch.Tensor):
        assert not ctx.needs_input_grad[1] and not ctx.needs_input_grad[2] and not ctx.needs_input_grad[3]
        input, weights_quantized, absmax, code = ctx.saved_tensors
        # grad_output: [*batch, out_features]
        weights_deq = dequantize_blockwise(weights_quantized, absmax=absmax, code=code)
        grad_input = grad_output @ weights_deq
        grad_bias = grad_output.flatten(0, -2).sum(dim=0) if ctx._has_bias else None
        return grad_input, None, None, None, grad_bias
 
 
class FrozenBNBEmbedding(nn.Module):
    def __init__(self, weight, absmax, code):
        super().__init__()
        self.num_embeddings, self.embedding_dim = weight.shape
        self.register_buffer("weight", weight.requires_grad_(False))
        self.register_buffer("absmax", absmax.requires_grad_(False))
        self.register_buffer("code", code.requires_grad_(False))
        self.adapter = None
 
    def forward(self, input, **kwargs):
        with torch.no_grad():
            # note: both quantuized weights and input indices are *not* differentiable
            weight_deq = dequantize_blockwise(self.weight, absmax=self.absmax, code=self.code)
            output = F.embedding(input, weight_deq, **kwargs)
        
        if self.adapter:
            output_cloned = torch.clone(output + self.adapter(input))
            return output_cloned
        else :
            return output
 
    @classmethod
    def from_embedding(cls, embedding: nn.Embedding) -> "FrozenBNBEmbedding":
        weights_int8, state = quantize_blockise_lowmemory(embedding.weight)
        return cls(weights_int8, *state)
 
    def __repr__(self):
        return f"{self.__class__.__name__}({self.num_embeddings}, {self.embedding_dim})"
 
 
def quantize_blockise_lowmemory(matrix: torch.Tensor, chunk_size: int = 2 ** 20):
    assert chunk_size % 4096 == 0
    code = None
    chunks = []
    absmaxes = []
    flat_tensor = matrix.view(-1)
    for i in range((matrix.numel() - 1) // chunk_size + 1):
        input_chunk = flat_tensor[i * chunk_size: (i + 1) * chunk_size].clone()
        quantized_chunk, (absmax_chunk, code) = quantize_blockwise(input_chunk, code=code)
        chunks.append(quantized_chunk)
        absmaxes.append(absmax_chunk)
 
    matrix_i8 = torch.cat(chunks).reshape_as(matrix)
    absmax = torch.cat(absmaxes)
    return matrix_i8, (absmax, code)
 
 
def convert_to_int8(model):
    """Convert linear and embedding modules to 8-bit with optional adapters"""
    for module in list(model.modules()):
        for name, child in module.named_children():
            if isinstance(child, nn.Linear):
                print(name, child)
                setattr(
                    module,
                    name,
                    FrozenBNBLinear(
                        weight=torch.zeros(child.out_features, child.in_features, dtype=torch.uint8),
                        absmax=torch.zeros((child.weight.numel() - 1) // 4096 + 1),
                        code=torch.zeros(256),
                        bias=child.bias,
                    ),
                )
            elif isinstance(child, nn.Embedding):
                setattr(
                    module,
                    name,
                    FrozenBNBEmbedding(
                        weight=torch.zeros(child.num_embeddings, child.embedding_dim, dtype=torch.uint8),
                        absmax=torch.zeros((child.weight.numel() - 1) // 4096 + 1),
                        code=torch.zeros(256),
                    )
                )

In [4]:
class GPTJBlock(transformers.models.gptj.modeling_gptj.GPTJBlock):
    def __init__(self, config):
        super().__init__(config)

        convert_to_int8(self.attn)
        convert_to_int8(self.mlp)


class GPTJModel(transformers.models.gptj.modeling_gptj.GPTJModel):
    def __init__(self, config):
        super().__init__(config)
        convert_to_int8(self)
        

class GPTJForCausalLM(transformers.models.gptj.modeling_gptj.GPTJForCausalLM):
    def __init__(self, config):
        super().__init__(config)
        convert_to_int8(self)


transformers.models.gptj.modeling_gptj.GPTJBlock = GPTJBlock

In [5]:
class T5ForConditionalGeneration(transformers.models.t5.modeling_t5.T5ForConditionalGeneration):
    def __init__(self, config):
        super().__init__(config)
        convert_to_int8(self)

transformers.models.t5.modeling_t5.T5ForConditionalGeneration = T5ForConditionalGeneration

In [6]:
config = transformers.GPTJConfig.from_pretrained("EleutherAI/gpt-j-6B")
tokenizer = transformers.AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

In [7]:
config.pad_token_id = config.eos_token_id
tokenizer.pad_token = config.pad_token_id

In [8]:
gpt = GPTJForCausalLM.from_pretrained("hivemind/gpt-j-6B-8bit", low_cpu_mem_usage=True)
#gpt = GPTJForCausalLM.from_pretrained("gustavecortal/fr-boris-8bit", low_cpu_mem_usage=True) French GPT-J Cedille's Boris

k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, bias=False)
fc_in Linear(in_features=4096, out_features=16384, bias=True)
fc_out Linear(in_features=16384, out_features=4096, bias=True)
k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, bias=False)
fc_in Linear(in_features=4096, out_features=16384, bias=True)
fc_out Linear(in_features=16384, out_features=4096, bias=True)
k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, 

In [9]:
if torch.cuda.is_available():  
    dev = "cuda:0" 
else:  
    dev = "cpu"  
device = torch.device(dev)  

gpt.to(device)

GPTJForCausalLM(
  (transformer): GPTJModel(
    (wte): FrozenBNBEmbedding(50400, 4096)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-27): 28 x GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): FrozenBNBLinear(4096, 4096)
          (v_proj): FrozenBNBLinear(4096, 4096)
          (q_proj): FrozenBNBLinear(4096, 4096)
          (out_proj): FrozenBNBLinear(4096, 4096)
        )
        (mlp): GPTJMLP(
          (fc_in): FrozenBNBLinear(4096, 16384)
          (fc_out): FrozenBNBLinear(16384, 4096)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): FrozenBNBLinear(4096, 50400)
)

In [29]:
import torch
import transformers
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from transformers import AutoTokenizer, AutoModelForCausalLM
import csv
# Load dataset
df = pd.read_csv('/kaggle/input/nlpgettingstarted/train.csv')
df = df[['text', 'target']]
df = df.rename(columns={'text': 'input_text', 'target': 'label'})
# Split dataset into training and test sets
#train_data, test_data = train_test_split(df, test_size=0.8, random_state=42)



In [30]:
# Split dataset into training, validation, and test sets
train_data, remaining_data = train_test_split(df, test_size=0.8, random_state=42)
validation_data, test_data = train_test_split(remaining_data, test_size=0.9, random_state=42)

# Check the sizes of each set
print("Training set size:", len(train_data))
print("Validation set size:", len(validation_data))
print("Test set size:", len(test_data))

Training set size: 6851
Validation set size: 114
Test set size: 648


In [31]:
# Train and evaluate model
train_data1 = train_data['input_text'].to_list()
y_train_data = train_data['label'].astype(str).to_list()
test_data = test_data[['input_text', 'label']]
y_validation_data = validation_data['label'].to_list()

# Encodage des données d'entraînement avec le tokenizer
model_inputs = tokenizer(train_data1, padding=True, truncation=True, max_length=128, return_tensors='pt').to(device)

In [32]:
df.head()

,input_text,label
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


# Few-Shot Prompting

In [42]:
predictions = []
for i in validation_data['input_text']:
    prompt_text = '''
 The following are tweets, their corresponding class (1 if it is about a real disaster, 0: if it is not about a real disaster) and the reason of classification.
Tweet: Gail and Russell saw lots of hail at their Dalroy home - they have video of twister 1/2 mile from their home #yyc http://t.co/3VfKEdGrsO Class: 1
Reason: The tweet mentions witnessing a twister near their home, which indicates a real weather event.

Tweet: real magic in real life: women went missing in Ohio town the focus of FBI probe after strange deaths and... http://t.co/6m0YNJWbc9 Class: 1
Reason: The tweet mentions missing women and an FBI investigation into strange deaths, suggesting a real-life event.

Tweet: Find out how your fund was used for Typhoon Haiyan in the Philippines. See @DevPeace Haiyan Relief Funds Report http://t.co/JwxrX1LsqO Class: 1
Reason: The tweet refers to a specific fund report and mentions Typhoon Haiyan, indicating a real disaster relief effort.

Tweet: Top insurer blasts lack of Australian Govt action on disaster mitigation http://t.co/sDgOUtWNtb via @smh Class: 1
Reason: The tweet discusses an insurer criticizing the lack of government action on disaster mitigation, suggesting a real situation involving disaster management.

Tweet: Photographer Brian Ruebs endures 4500-feet climb to capture bride and groom http://t.co/BmWmpOyDIg Class: 0
Reason: The tweet mentions a photographer climbing to capture a bride and groom, which is not indicative of a real disaster or event.

Tweet: Thank you so so much to everyone for posting the rain and hail outside ... I had no idea guys ???????? Class: 0
Reason: The tweet expresses gratitude for people posting about rain and hail outside, but it doesn't indicate a real disaster or event.

Tweet: Me trying to pass lax with my family ends up by me having to run after the ball after it gets bombed over my head Class: 0
Reason: The tweet describes a personal experience in a game and doesn't refer to a real disaster or event.

Tweet: Beautiful disaster // Jon McLaughlin is such a good song Class: 0
Reason: The tweet mentions a song titled "Beautiful Disaster" by Jon McLaughlin, indicating a subjective opinion about a song and not a real disaster.

Tweet: The result of failure in correcting structural problems with West's financial markets collapse https://t.co/DvieABlOFz Class: 1
Reason: The tweet mentions the collapse of West's financial markets due to the failure to correct structural problems, indicating a real financial event.

Tweet: #ICYMI #Announcement from Al Jackson... http://t.co/7BevuJE5eP Class: 0
Reason: The tweet refers to an announcement from Al Jackson, but it doesn't provide any context or information about a real disaster or event.

Tweet: @CorleoneDaBoss bc its risky and costly I don't see a need to do that when there isn't anything there. It's totally desolate. Class: 1
Reason: The tweet mentions the risk and cost associated with a certain action, suggesting a real situation involving a desolate place.

Tweet:screams http://t.co/PU7C4Hhbxj Class: 0
Reason: The tweet consists of a simple exclamation and a link, without providing any context or information about a real disaster or event.

Tweet: #news #science London warship exploded in 1665 because sailors were recycling artillery cartridges... http://t.co/r4WGXrA59M #life #tech Class: 1
Reason: The tweet mentions a historical event where a London warship exploded due to the recycling of artillery cartridges, indicating a real incident.

Tweet:?? Yes, I do have 2 guns ?? ?? Class: 0
Reason: The tweet mentions the possession of guns but doesn't provide any context or information about a real disaster or event.

Tweet: Four Technologies That Could Let Humans Survive Environmental Disaster - http://t.co/4RTpJrHsqe Class: 0
Reason: The tweet refers to an article discussing technologies that could help humans survive environmental disasters, but it doesn't indicate a real disaster itself.

Tweet: Hotel evacuated after fire on Shore Drive in Norfolk http://t.co/6X0xHlbxji Class: 1
Reason: The tweet mentions a hotel being evacuated after a fire on Shore Drive in Norfolk, indicating a real incident of a fire and evacuation.

Tweet: I waited 2.5 hours to get a cab my feet are bleeding Class: 1
Reason: The tweet mentions waiting for a long time to get a cab and experiencing bleeding feet, indicating a real situation of waiting and physical discomfort.

Tweet: @_301DC @Cloudy_goldrush i hate white people mo Class: 1
Reason: The tweet expresses hatred towards a specific racial group, indicating a negative sentiment and potential real-life issue related to racism.

Tweet: the new quest type is 'level up quest'. its an always present quest with x2 exp designed to help people level up outside of emergency quests Class: 0
Reason: The tweet refers to a game quest with double experience points, indicating a game-related feature and not a real disaster or event.

Tweet: The Catastrophic Effects of Hiroshima and Nagasaki Atomic Bombings Still Being Felt Today http://t.co/QVlxpyyyCd Class: 1
Reason: The tweet mentions the long-term effects of the atomic bombings in Hiroshima and Nagasaki, indicating a real historical event and its ongoing impact.

Tweet: To All The Meat-Loving Feminists Of The World Riot Grill Has Arrived http://t.co/um3wTL5r7K #arts http://t.co/2LQyxZQ5DN Class: 0
Reason: The tweet refers to a meat-loving feminist event, indicating a specific topic related to food and feminism, but it doesn't involve a real disaster or event.

Tweet: My mic and controllers aren't working one second Class: 0
Reason: The tweet mentions a personal issue with a microphone and controllers not working, but it doesn't indicate a real disaster or event.

Tweet: Men escape car engulfed in flames in Parley's Canyon crews investigating cause - http://t.co/zevAn9kJzL http://t.co/UUZFs1L5Kt Class: 1
Reason: The tweet describes men escaping from a car engulfed in flames in Parley's Canyon, indicating a real incident of a car fire and rescue operation.

Tweet: #flood #disaster Burst Water Pipe Floods Apartments at NYCHA Senior Center - NY1: NY1Burst Water Pipe Floods A... http://t.co/w7SIIdujOH Class: 1
Reason: The tweet mentions a burst water pipe causing flooding in apartments at an NYCHA Senior Center, indicating a real flood-related incident.

Tweet: A Tale of Two Pox - Body Horrors http://t.co/W2IXT1k0AB #virus #infectiousdiseases #bioterrorism Class:1
Reason: The tweet refers to a blog post titled "A Tale of Two Pox" discussing viruses, infectious diseases, and bioterrorism, indicating a real topic related to public health and diseases. 

Tweet: 320 [IR] ICEMOON [AFTERSHOCK] | http://t.co/M4JDZMGJoW | @djicemoon | #Dubstep #TrapMusic #DnB #EDM #Dance #IcesÛ_ http://t.co/n0uhAsfkBv Class: 0
Reason: The tweet mentions a music-related content and hashtags, indicating a promotional post about a DJ and music genres, but it doesn't involve a real disaster or event.

Tweet: Arson suspect linked to 30 fires caught in Northern California http://t.co/mmGsyAHDzb Class: 1
Reason: The tweet mentions an arson suspect linked to 30 fires being caught in Northern California, indicating a real incident related to arson and law enforcement.

Tweet: Elecman could electrocute me and I'd say thanks. Class: 0
Reason: The tweet expresses a personal opinion about Elecman and being grateful for potential electrocution, but it doesn't indicate a real disaster or event.

Tweet: Growth dries up for BHP Billiton as oil price collapse bites http://t.co/HQoD6v6DnC Class: 0
Reason: The tweet refers to the growth decline of BHP Billiton due to the collapse of oil prices, indicating a business-related news but not a real disaster or event.

Tweet: 73rd GOODE Water Ski National Championships will go on as planned next week http://t.co/PgKBT3MBAp. (Event w/ damage from a tornado on Mon) Class: 1
Reason: The tweet mentions the 73rd GOODE Water Ski National Championships proceeding as planned despite damage from a tornado, indicating a real event affected by a tornado.

Tweet: Wreckage 'conclusively confirmed' as from MH370: Malaysia PM http://t.co/Rp2DxFKHDQ | https://t.co/akmIHLRIs1 Class:1
Reason: The tweet mentions the wreckage of MH370 being conclusively confirmed by the Malaysia PM, indicating a real incident related to the missing Malaysian Airlines flight MH370.

Tweet: I liked a @YouTube video from @chaboyyhd http://t.co/Yr67ugEsrm Battlefield 4 Funny Moments - Dukes of Hazard Undercover Soldier Class: 0
Reason: The tweet indicates liking a YouTube video about funny moments in Battlefield 4, indicating a personal preference for gaming content but not a real disaster or event.

Tweet:Just down drown me k I can't swim https://t.co/sJoEing76t Class: 0
Reason: The tweet mentions not being able to swim and using the phrase "just don drown me," but it doesn't indicate a real disaster or event.

Tweet: U.S.PACIFIC COMMAND.
I can see it!
They gave their all in the peace unity festival
It disappears when freedom
A Violent Storm hit Sea Class: 1
Reason: The tweet mentions a violent storm hitting the sea, indicating a real incident related to a storm or natural disaster.

Tweet: Typhoon Soudelor taking dead aim at Taiwan http://t.co/BhsUxVq6NF Class: 1
Reason: The tweet mentions Typhoon Soudelor heading towards Taiwan, indicating a real weather event and the potential impact of a typhoon.

Tweet: @BenKin97 @Mili_5499 remember when u were up like 4-0 and blew it in one game? U probs don't because it was before the kings won the cup Class: 1
Reason: The tweet mentions a sports-related event where someone blew a lead in a game, referring to a past event before the Kings won the cup.

Tweet: Officials: Alabama home quarantined over possible ... http://t.co/Rb2s0jmleJ Class: 1 
Reason: The tweet mentions a home being quarantined, which indicates a possible disaster situation.

Tweet: @Grazed @invalid @Stretcher @Rexyy @Towel 'Ben favorited' Class: 0 
Reason: The tweet does not mention any disaster or indicate a potential disaster.

Tweet: Two giant cranes holding a bridge collapse into nearby homes http://t.co/9asc1hhFNJ Class: 1 
Reason: The tweet describes a bridge collapse, which is a disaster event.

Tweet: Riot police intervene after Southampton and Vitesse Arnhem supporters clash: Û¢ Fans clash in buildup to second... http://t.co/sKVNmtZGeG Class: 1 
Reason: The tweet mentions supporters clashing, indicating a potential disaster situation.

Tweet: Families to sue over Legionnaires: More than 40 families affected by the fatal outbreak of Legionnaires' disea... http://t.co/3sNyOOhseq Class: 1 
Reason: The tweet mentions a fatal outbreak of Legionnaires' disease, which is a disaster event.

Tweet: Downtown Emergency Service Center is hiring! #Chemical #Dependency Counselor or Intern in #Seattle apply now! #jobs http://t.co/SKQPWSNOin Class: 0 
Reason: The tweet is about a job opening and does not mention any disaster.

Tweet: Condemnation clearly replacing the latest response aimlessly dryer evacuate detersion de: HLg Class: 1 
Reason: The tweet uses terms like "evacuate" and "condemnation," indicating a potential disaster situation.

Tweet: Perhaps the criminal murderous #nazis should pack their bags & evacuate themselves from London & all #UK? #TubeStrike WELL DONE! Class: 0 
Reason: The tweet mentions the #TubeStrike but in a sarcastic context and does not indicate an actual disaster.

Tweet: #np agalloch - the desolation song Class: 0 
Reason: The tweet is about a song and does not mention any disaster.

Tweet: New Ladies Shoulder Tote Handbag Women Cross Body Bag Faux Leather Fashion Purse - Full reÛ_ http://t.co/BLAAWHYScT http://t.co/dDR0zjXVQN Class: 0
Reason: The tweet is about a fashion item and does not mention any disaster.

Tweet:Severe Thunderstorm Warning including Midwest City OK Del City OK Choctaw OK until 10:30 PM CDT http://t.co/ogxSY4GWD1 Class: 1 
Reason: The tweet mentions a severe thunderstorm warning, indicating a potential disaster situation.

Tweet: Kach was a group to which belonged Baruch Goldstein a mass murderer who in 1994 shot and killed 29 PalestinianÛ_ http://t.co/bXGNQ57xvb Class: 1 
Reason: The tweet mentions a mass murderer and the killing of Palestinians, indicating a disaster event.

Tweet: Rly tragedy in MP: Some live to recount horror: ÛÏWhen I saw coaches of my train plunging into water I called ... http://t.co/CaR5QEUVHH Class: 1 
Reason: The tweet describes a train tragedy with coaches plunging into water, indicating a disaster event.

Tweet: {} Class:'''.format(i)
    prompt = tokenizer(prompt_text, truncation=True, return_tensors='pt')
    prompt = {key: value.to(device) for key, value in prompt.items()}

    out = gpt.generate(**prompt, max_length=1, top_k=40, top_p=0.9, temperature=0.3, do_sample=False, repetition_penalty=1.1, num_beams=1, pad_token_id=0)
    y_hat = tokenizer.decode(out[0][-1]).strip()
    predictions.append(int(y_hat) if y_hat.isdigit() else None)

Input length of input_ids is 2048, but `max_length` is set to 1. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Input length of input_ids is 2048, but `max_length` is set to 1. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Input length of input_ids is 2048, but `max_length` is set to 1. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Input length of input_ids is 2048, but `max_length` is set to 1. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Input length of input_ids is 2048, but `max_length` is set to 1. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Input length of input_ids is 2048, but `max_length` is set to 1. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Input length of input_ids is 2048, but `max_length` is set to 1. This can lead to unexpe

In [43]:
# Define evaluation function
def evaluate(prediction, target):
    from sklearn.metrics import accuracy_score
    return accuracy_score(prediction, target)

In [45]:
evaluate(predictions,y_validation_data)

0.6052631578947368

In [ ]:
# See where the model failed
df_evaluation.loc[predictions != y, 'text'].values